In [ ]:
# numpy module for numeric data processing and vectorization
import numpy as np

# pandas module for opening dataset & processing it
import pandas as pd

import re
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
plt.style.use('ggplot')
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline


In [ ]:
df = pd.read_json("/content/drive/MyDrive/Sarcasm_Headlines_Dataset_v2.json", lines=True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [ ]:
df['headline'][49]

'god getting strong urge to bring back dinosaurs'

In [ ]:
doc = nlp(df['headline'][49])

In [ ]:
for tok in doc:
  print(tok.text, "...", tok.dep_)

god ... nsubj
getting ... ROOT
strong ... amod
urge ... dobj
to ... aux
bring ... acl
back ... prt
dinosaurs ... dobj


In [ ]:
def get_entities(sent):
  sub = ""
  obj = ""

  prv_tok_dep = ""    
  prv_tok_text = ""   

  prefix = ""
  modifier = ""
  
  for tok in nlp(sent):
    if tok.dep_ != "punct":
      if tok.dep_ == "compound":
        prefix = tok.text
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      if tok.dep_.find("subj") == True:
        sub = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      if tok.dep_.find("obj") == True:
        obj = modifier +" "+ prefix +" "+ tok.text
        
      
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text

  return [sub.strip(), obj.strip()]


In [ ]:
get_entities("The paper has more than 400 citations.")

['paper', '400  citations']

In [ ]:
entity_pairs = []
for i in tqdm(df['headline']):
  entity_pairs.append(get_entities(i))


100%|██████████| 28619/28619 [06:26<00:00, 73.99it/s]


In [ ]:
entity_pairs[10:20]

[['lesbian', 'amazing  that'],
 ['amanda peet', 'daughter'],
 ['', 'current  ebola'],
 ['hillary clinton', 'boko schoolgirls'],
 ['new  that', 'purely  gasoline'],
 ['', 'economic advisory council'],
 ['area boy', 'jumping  phase'],
 ['area man', 'gurney'],
 ['', 'disabilities'],
 ['lin manuel miranda', 'phone']]

In [ ]:
def get_relation(sent):
  doc = nlp(sent)
  matcher = Matcher(nlp.vocab)
  
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1", None, pattern) 
  matches = matcher(doc)
  k = len(matches) - 1
  span = doc[matches[k][1]:matches[k][2]] 
  return(span.text)

In [ ]:
get_relation("Mohammad completed his assignment.")

'completed'

In [ ]:
relations = [get_relation(i) for i in tqdm(df['headline'])]

100%|██████████| 28619/28619 [05:27<00:00, 87.48it/s]


In [ ]:
pd.Series(relations).value_counts()[120:130]

set           26
recommends    26
plan          26
warn          25
put           25
use           25
passes        24
wait          24
read          23
decides       23
dtype: int64

In [ ]:
# Extracting the subject
source = [i[0] for i in entity_pairs]

# Extracting the object
target = [i[1] for i in entity_pairs]

kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relations})

In [ ]:
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

In [ ]:
G=nx.from_pandas_edgelist(kg_df[kg_df['edge']=="read"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes
nx.draw(G, with_labels=True, node_color='yellow', node_size=900, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()
